In [21]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from torch.autograd import Variable
from numpy.fft import rfft2, irfft2
from sklearn import svm
import cv2
from matplotlib import pyplot as plt
import joblib
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [131]:
class CNN(nn.Module):
    def __init__(self, in_channels = 1, num_classes = 10):
        super(CNN, self).__init__()
        self.C1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(3,3), stride=(2,2))
        self.C2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(2,2), padding=(1,1))
        self.ave_pool = nn.AvgPool2d(3,1)
        self.fc1 = nn.Linear(400, 400)
        self.fc2 = nn.Linear(400, num_classes)

    def forward(self, x):
        x = nn.functional.relu(self.C1(x))
        x = self.pool(x)
        x = nn.functional.relu(self.C2(x))
        x = self.ave_pool(x)
        x = x.reshape(x.shape[0],-1)
        #x = x.view(-1, 16*5*5)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.sigmoid(self.fc2(x))
        
        return x


In [124]:
learning_rate = 0.001
num_classes = 10
batch_size = 256
num_epochs = 100

In [132]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [126]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


In [146]:
for epoch in range(num_epochs):
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)
        

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        

In [147]:
def accuracy(loader, model):
    if loader.dataset.train:
        print("---Trainning Accuracy---")
    else:
        print("---Test Accuracy---")
    
    num_correct = 0
    num_sample = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions==y).sum()
            num_sample  += predictions.size(0)
        print(f"Got {num_correct}/{num_sample} with accuracy {round(float(num_correct/num_sample),4)}")
    model.train()


In [148]:
accuracy(train_loader,model)
accuracy(test_loader, model)

---Trainning Accuracy---
Got 59874/60000 with accuracy 0.9979
---Test Accuracy---
Got 9899/10000 with accuracy 0.9899


In [223]:
# predict
def predict_(test_loader, model):
    predict = []
    true_label = []
    n = np.random.randint(len(test_loader))
    i = 0
    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(device=device)
            true_label = y
            y = y.to(device=device)
            scores = model(x)
            _, predictions = scores.max(1)
        
            predict = predictions.to('cpu')
            i+=1
            if i == n:
                break
    return predict, true_label


In [270]:
#Predict 
predict, true_label = predict_(test_loader, model)
predict = [np.array(i,dtype=int) for i in predict]
true_label = [np.array(i,dtype=int) for i in true_label]
t = np.random.randint(len(predict))
print("---Random Predict---")
print('|  Number predict  |')
print('____________________')
print("|"," "*6 ,predict[t], " "* 7, "|")
print('____________________')
print('|True number label |')
print('____________________')
print("|"," "*6, true_label[t]," "*7, "|")
print('____________________')
print("        -------Wrong Predict-------")
print("____________________________________________")
print("| Predict Number    |  True Label's Number |")
print("____________________________________________")
for i in range(256):
    if predict[i]!=true_label[i]:
        print("|       ",predict[i], "  " *4,"|"," "* 9, true_label[i], " "* 8,"|")
        print("____________________________________________")



---Random Predict---
|  Number predict  |
____________________
|        7         |
____________________
|True number label |
____________________
|        7         |
____________________
        -------Wrong Predict-------
____________________________________________
| Predict Number    |  True Label's Number |
____________________________________________
|        3          |           9          |
____________________________________________
|        7          |           2          |
____________________________________________
|        7          |           1          |
____________________________________________
|        7          |           0          |
____________________________________________
|        3          |           5          |
____________________________________________
